In [5]:
fname = '../tests/test_files/review1.html'
with open(fname,'r') as f:
    html = f.read()

In [6]:
from BeautifulSoup import BeautifulSoup
import sqlite3

In [7]:
class Review(object):
    def __init__(self, soup, current):
        self.soup = soup
        self.attr = {}
        
        self.attr.update(current)
        
        self.attr['Rating'] = self.get_rating()
        self.attr['Review'] = self.get_review()
        self.attr['ReviewDate'] = self.get_reviewdate()
        self.attr['VisitDate'] = self.get_visitdate()
        self.attr['Title'] = self.get_title()
        self.attr['UID'] = self.get_uid()
        self.attr['ReviewID'] = self.get_reviewID()
        self.attr['User'] = self.get_user()
        self.attr['UserHome'] = self.get_userhome()
        self.attr['Key'] = self.attr['UID'] + '-' + str(self.attr['ReviewID'])        
        
    def get_rating(self):        
        tag = self.soup.find(lambda tag: tag.name == 'img' and tag.has_key('alt') and 'stars' in tag['alt'])
        return int(tag['alt'][0])
    
    def get_review(self):
        texts = self.soup.findAll('div',{'class':'entry'})
        return texts[-1].text
    
    def get_reviewdate(self):
        txt = self.soup.find("span", {"class":"ratingDate"}).contents[0]
        date = txt.replace('Reviewed','').strip()
        return date #date_object.strftime('%m-%d-%Y')

    def get_visitdate(self):
        try:
            txt = self.soup.find('div',{'class':'recommend'}).text
        except:
            txt = ''
        return txt.replace('Visited','').strip()
    
    def get_title(self):
        return self.soup.find('span',{'class':'noQuotes'}).text
    
    def get_uid(self):
        tag = self.soup.find('div',{'class':'member_info'})
        uid = tag.find(lambda tag: tag.name=='div' and tag.has_key('id'))['id']
        return uid[4:uid.find('-')]
    
    def get_reviewID(self):
        return int(self.soup['id'][7:])

    def get_user(self):
        try:
            return raw_review.find('div',{'class':'username mo'}).text
        except:
            return ''    
        
    def get_userhome(self):
        try:
            return self.soup.find('div',{'class':'location'}).text
        except:
            return ''
    
    def get_vars(self):
        return self.attr.keys()
    
    def get_values(self):
        return self.attr.values()
    



In [8]:
class ReviewList(object):
    
    def __init__(self):
        self.reviews = []
        
    def size(self):
        return len(self.reviews)
    
    def DBdump(self, dbname, tbl_schema, tablename):
        with sqlite3.connect(dbname) as conn:
            conn.execute(tbl_schema)
            for item in self.reviews:
                qmarks = ', '.join('?' * len(item.get_vars()))
                qry = "INSERT OR IGNORE INTO {} {} VALUES ({})".format(tablename, tuple(item.get_vars()), qmarks)
                conn.execute(qry, item.get_values()) 
            conn.commit()
    
    def append(self, item):
        self.reviews.append(item)
        

In [9]:
VietnamDB = """
    CREATE TABLE IF NOT EXISTS Vietnam (
        Key TEXT PRIMARY KEY,
        Country TEXT,
        ItemReviewed TEXT,
        Type TEXT,
        ParentGroup TEXT,
        Location TEXT,
        Rating INT,
        Review TEXT,
        ReviewDate TEXT,
        VisitDate TEXT,
        Title TEXT,
        UID INT,
        ReviewID TEXT,
        User TEXT,
        UserHome TEXT
        )"""

    
soup = BeautifulSoup(html)

current = {}
addr = soup.find('h1',{'class':'header   '}).text.split(',')

current['ItemReviewed'] = addr[0].strip()
current['Location'] = addr[1].strip()
current['Country'] = 'Vietnam'
current['Type'] = 'things to do'
current['ParentGroup'] = ''

tablename = 'Vietnam'


mylist = ReviewList()
tags = soup.find('div',{'id':'REVIEWS'}).findChildren(recursive=False)
reviews = filter(lambda tag: tag.has_key('id') and 'review_' in tag['id'], tags)
for idx, raw_review in enumerate(reviews):
    print idx
    obj = Review(raw_review, current)
    mylist.append(obj)

mylist.DBdump('TripAdvisor.db', VietnamDB, tablename)

0
1
2
3
4
5
6
7
8
9


In [381]:
pp = pprint.PrettyPrinter(indent=4)
for item in mylist.reviews:
#    pp.pprint(item.attr)


IndentationError: expected an indented block (<ipython-input-381-b11384db51b8>, line 3)

In [25]:
qry = "SELECT COUNT(*) FROM Vietnam"
with sqlite3.connect('TripAdvisor.db') as conn:
    cur = conn.cursor()
    cur.execute(qry)
    result = cur.fetchall()

In [24]:
map(lambda x: x[0], result)

[5, 5, 5, 5, 4, 3, 5, 5, 5, 5]

In [28]:
result[0][0]

10